In [1]:
import scipy as sp
import numpy as np
import os 
import pickle

import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

from architectures.cycleGAN_u_net import *

In [2]:
task='TRAIN'
#task='TEST'
SEED=1

PATH = 'cycleGAN_test'

In [3]:
test_size = 5000
validating_size=1000

LEARNING_RATE = 2e-4
BETA1 = 0.5
BATCH_SIZE = 1
EPOCHS = 1
SAVE_SAMPLE_PERIOD = 50
SEED = 1
preprocess=False
cost_type='GAN'

cycl_weight=10.
discr_steps=1
gen_steps=4

ndf = 4
ngf = 8
s=2
f = 2
d=0.8

stddev_d=0.02
stddev_g=0.02

global d_sizes, g_sizes

g_sizes_enc={
    
        'conv_layers': [
                            (ngf/2, f*16, s,  False, 1, lrelu,  tf.glorot_normal_initializer()), #(batch, 52, 64, 1) =>  (batch, 26, 32, ngf)
                            (ngf,   f*8, s,  False,  1, lrelu,  tf.glorot_normal_initializer()), #(batch, 52, 64, 1) =>  (batch, 26, 32, ngf)
                            (ngf*2, f*6, s, 'bn',    d, lrelu,  tf.glorot_normal_initializer()),#(batch, 26, 32, ngf) => (batch, 13, 16, ngf*2)
                            (ngf*4, f*4, s, 'bn',    d, lrelu,  tf.glorot_normal_initializer()),#(batch, 13, 16, ngf*4) => (batch, 7, 8, ngf*4)
                            (ngf*8, f*2, s, 'bn',    d, lrelu,  tf.glorot_normal_initializer()),#(batch, 7, 8, ngf*4) => (batch, 4, 4, ngf*4)
                            (ngf*8, f,   1, 'bn',    d, lrelu,  tf.glorot_normal_initializer()),#(batch, 4, 4, ngf*4) => (batch, 2, 2, ngf*4)
                           #(ngf*8, f, s, 'bn', 1, lrelu, tf.truncated_normal_initializer(stddev_g)),#(batch, 2, 2, ngf*4) => (batch, 1, 1, ngf*4)
                        
                       ],
}

g_sizes_dec={

     
     'deconv_layers': [

                        (ngf*8, f,   1,   'bn', d, tf.nn.relu,    tf.glorot_normal_initializer()),#(batch, 1, 1, ngf*4) => (batch, 2, 2, ngf*4*2)
                        (ngf*4, f*2, s,   'bn', d, tf.nn.relu,    tf.glorot_normal_initializer()),#(batch, 2, 2, ngf*4*2) => (batch, 4, 4, ngf*4*2)
                        (ngf*2, f*4, s,   'bn', d, tf.nn.relu,    tf.glorot_normal_initializer()),#(batch, 4, 4, ngf*4*2) => (batch, 7, 8, ngf*4*2)
                        (ngf,   f*6, s,   'bn', d, tf.nn.relu,    tf.glorot_normal_initializer()),#(batch, 7, 8, ngf*4*2) => (batch, 13, 16, ngf*2*2)
                        (ngf/2, f*16,s,  'bn', 1,  tf.nn.relu,    tf.glorot_normal_initializer()), #(batch, 52, 64, 1) =>  (batch, 26, 32, ngf)
                        (3,     f*16,s, False, 1,  tf.sigmoid,    tf.glorot_normal_initializer()),#(batch, 26, 32, ngf*2) => (batch, 52, 64, 1)
                   
                     ],  
}


d_sizes={
    
     'conv_layers': [
                         (ndf/2, f*4, s*2,  False, 1, lrelu, tf.truncated_normal_initializer(stddev_d)), #(batch, 52, 64, 2) => (batch, 26, 32, ndf)
                         (ndf,   f*2, s,   'bn',   d, lrelu, tf.truncated_normal_initializer(stddev_d)), #(batch, 26, 32, ndf) => (batch, 13, 16, ndf*2)
                         (ndf*2, f,   s,   'bn',   d, lrelu, tf.truncated_normal_initializer(stddev_d)), #(batch, 13, 16, ndf*2) => (batch, 7, 8, ndf*4)
                         (ndf*4, f,   s,   'bn',   1, lrelu, tf.truncated_normal_initializer(stddev_d)), #(batch, 7, 8, ndf*4) => (batch, 7, 8, ndf*8)
                        #(ndf*8, f, 1, 'bn', 1, lrelu, tf.truncated_normal_initializer(stddev_d)), #(batch, 7, 8, ndf*8) => (batch, 7, 8, ndf*8)
                         
                    ],
                    
     #'readout_conv_layer':[
     #                    (f,   s,  False,   d,  tf.truncated_normal_initializer(stddev_d) )
     #],
     'dense_layers': [(ndf*32, 'bn',  d, lrelu, tf.truncated_normal_initializer(stddev_d)),
                     (ndf,     False, d, lrelu, tf.truncated_normal_initializer(stddev_d))],
     
     'readout_layer_w_init':tf.truncated_normal_initializer(stddev_d),
}


In [4]:
def Horse2Zebra():
    
    train_A = np.array(
        [plt.imread("apple2orange/trainA/"+filename) for filename in os.listdir("apple2orange/trainA")]
    )
    
    train_B = np.array(
        [plt.imread("apple2orange/trainB/"+filename) for filename in os.listdir("apple2orange/trainB")]
    )
    
    m = np.minimum(train_A.shape[0],train_B.shape[0])
    
    _, n_H, n_W, n_C = train_A.shape
    
    X_train_A = train_A[0:m]/256
    X_train_B = train_B[0:m]/256
    
    #train_A=train_A.astype(np.float32)
    #mean_A=np.mean(train_A,axis=0)
    #train_A_1=train_A-mean_A
    #std_A=np.std(train_A_1,axis=0)
    #train_A_2=train_A_1/std_A
    #
    #train_A=train_A_2
    #
    #train_B=train_B.astype(np.float32)
    #mean_B=np.mean(train_B,axis=0)
    #train_B_1=train_B-mean_B
    #std_B=np.std(train_B_1,axis=0)
    #train_B_2=train_B_1/std_B
    #
    #train_B = train_B_2
    
    tf.reset_default_graph()

    gan = cycleGAN_u_net(n_H_A=n_H, n_W_A=n_W,
                         n_H_B=n_H, n_W_B=n_W, n_C=n_C, 
                         mean_A=mean_A, std_A=std_A, 
                         mean_B=mean_B, std_B=std_B, 
                         d_sizes_A=d_sizes, d_sizes_B=d_sizes,
                         g_sizes_enc_A=g_sizes_enc, g_sizes_enc_B= g_sizes_enc,
                         g_sizes_dec_A=g_sizes_dec, g_sizes_dec_B= g_sizes_dec,
                         lr=LEARNING_RATE,beta1=BETA1, preprocess=False,
                         cost_type='GAN', gan_weight=1, cycl_weight=cycl_weight,
                         discr_steps=discr_steps, gen_steps=gen_steps,
                         batch_size=BATCH_SIZE, epochs=EPOCHS,
                         save_sample=SAVE_SAMPLE_PERIOD, path=PATH, seed=SEED)
    
    vars_to_train= tf.trainable_variables()
    
    if task == 'TRAIN':
        init_op = tf.global_variables_initializer()
        
    if task == 'TEST':
        vars_all = tf.global_variables()
        vars_to_init = list(set(vars_all)-set(vars_to_train))
        init_op = tf.variables_initializer(vars_to_init)
        
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()    
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
    
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
        
        sess.run(init_op)

        if task=='TRAIN':
            print('\n Training...')
            
            if os.path.exists(PATH+'/'+PATH+'.ckpt.index'):
                saver.restore(sess,PATH+'/'+PATH+'.ckpt')
                print('Model restored.')
            
            gan.set_session(sess)
            unact_A, unact_B = gan.fit(X_train_A, X_train_B, 1)
            
            save_path = saver.save(sess, PATH+'/'+PATH+'.ckpt')
            print("Model saved in path: %s" % save_path)
        
        if task=='TEST':
            print('\n Evaluate model on test set...')
            saver.restore(sess,PATH+'/'+PATH+'.ckpt')
            print('Model restored.')
            
            gan.set_session(sess) 
            
        done = True
        while not done:
            
            
            j = np.random.choice(len(X_train_A))
            true_img = X_train_A[j]
            sample_img = gan.get_sample(true_img)
            
            plt.subplot(1,2,1)
            plt.imshow(true_img.reshape(n_H,n_W),cmap='gray')
            plt.subplot(1,2,2)
            plt.imshow(sample_img.reshape(n_H,n_W),cmap='gray')
            
            fig=plt.gcf()
            fig.set_size_inches(5,8)
            plt.savefig(PATH+'/sample_{0}_at_iter_{1}.png'.format(j, total_iters),dpi=300)
            
            ans = input("Generate another?")
            if ans and ans[0] in ('n' or 'N'):
                done = True
                
    return unact_A, unact_B

In [ ]:
if __name__=='__main__':

    if task == 'TRAIN':
        if not os.path.exists(PATH):
            os.mkdir(PATH)
    
        elif os.path.exists(PATH):
            if os.path.exists(PATH+'/checkpoint'):
                ans = input('A previous checkpoint already exists, choose the action to perform \n \n 1) Overwrite the current model saved at '+PATH+'/checkpoint \n 2) Start training a new model \n 3) Restore and continue training the previous model \n ')
                
                if ans == '1':
                    print('Overwriting existing model in '+PATH)
                    for file in os.listdir(PATH):
                        file_path = os.path.join(PATH, file)
                        try:
                            if os.path.isfile(file_path):
                                os.unlink(file_path)
                            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
                        except Exception as e:
                            print(e)
                            
                elif ans == '2':
                    PATH = input('Specify the name of the model, a new directory will be created.\n')
                    os.mkdir(PATH)    
        
        unact_A, unact_B = Horse2Zebra()
   
    elif task == 'TEST': 
        if not os.path.exists(PATH+'/checkpoint'):
            print('No checkpoint to test')
        else:
            Horse2Zebra()

Convolutional Network architecture detected for discriminator A
Convolutional Network architecture detected for discriminator B
Generator_A_to_B
Input for generator encoder shape (?, 256, 256, 3)
Output of generator encoder, 
 and input for generator decoder shape (?, 1, 1, 4096)
Generator output shape (?, 256, 256, 3)
Generator_B_to_A
Input for generator encoder shape (?, 256, 256, 3)
Output of generator encoder, 
 and input for generator decoder shape (?, 1, 1, 4096)
Generator output shape (?, 256, 256, 3)
Generator_B_to_A
Input for generator encoder shape (?, 256, 256, 3)
Output of generator encoder, 
 and input for generator decoder shape (?, 1, 1, 4096)
Generator output shape (?, 256, 256, 3)
Generator_B_to_A
Input for generator encoder shape (?, 256, 256, 3)
Output of generator encoder, 
 and input for generator decoder shape (?, 1, 1, 4096)
Generator output shape (?, 256, 256, 3)
Generator_A_to_B
Input for generator encoder shape (?, 256, 256, 3)
Output of generator encoder, 
 a

In [ ]:
plt.imshow(unact_A[0])